In [1]:
import os
from llama_cpp import Llama
import datasets
from tqdm.notebook import tqdm

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
import time

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
llm = Llama(
    model_path="../quantize_model/llama3-8b-instruct-Q4K_M.gguf",
    n_gpu_layers=-1,   # offload all layers to GPU
    n_ctx=4096,
    temperature=0.0,
    verbose=False
)

llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


In [4]:
%%time
output = llm("Why is the sky blue?", max_tokens=256)
print(output["choices"][0]["text"])

 – A simple explanation
Have you ever looked up at the sky and wondered why it appears blue? It’s a question that has puzzled people for centuries, and the answer is actually quite simple.
The sky appears blue because of a phenomenon called scattering. When sunlight enters Earth’s atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century. The scattering of light by the tiny molecules in the atmosphere is responsible for the blue color of the sky.
Here’s a simple way to visualize it:
Imagine you are shining a flashlight through a prism, which is a transparent material that bends light. The light will be refracted, or bent, as it passes through the prism, and you will see a rainbow-like spect

## Test performance

In [5]:
dataset = datasets.load_dataset('../dataset/thai-sentiment/')

In [6]:
df = dataset['test'].to_pandas()

In [7]:
df

,label,text
0,0,ไปเชียงใหม่ก็ระวังตัวกันเองนะครับ ต่อให้ขับมาแ...
1,0,อิเหี้ยจัยเย็น มันอินนิสฟรีเกาหลีว่ะ...
2,0,ไปกินมาไม่เห็นเหมือนในรูปเลย
3,0,คือค่าเหล้าเมื่อวานเปิดบลูเลเบลได้ขวดนึงเลยอะ ...
4,0,อะไรคือการกินสเมอร์นอฟรสใหม่ที่รสชาติเหมือนยาแ...
...,...,...
195,0,แอ่กกก
196,0,มันเว่อร์ แค่โฆษณาผ้าอนามัย
197,0,เผ็ดกูกินไม่ได้ !!!
198,0,หนองเมื่อทำให้สุกแล้วกินได้ใช่ไหม?! . . . . . ...


In [8]:
sample_df = df[:10]

In [9]:
sample_df

,label,text
0,0,ไปเชียงใหม่ก็ระวังตัวกันเองนะครับ ต่อให้ขับมาแ...
1,0,อิเหี้ยจัยเย็น มันอินนิสฟรีเกาหลีว่ะ...
2,0,ไปกินมาไม่เห็นเหมือนในรูปเลย
3,0,คือค่าเหล้าเมื่อวานเปิดบลูเลเบลได้ขวดนึงเลยอะ ...
4,0,อะไรคือการกินสเมอร์นอฟรสใหม่ที่รสชาติเหมือนยาแ...
5,1,ตามดันๆๆ
6,0,เออ ข่าวบาบีก้อนอะที่เค้าบอกกันว่าถ้าสุกก็กินไ...
7,1,กลุ่มนี้ รีโว่ คุ้มสุดครับ กำลังดีสุด แรงม้าไม...
8,0,ข่าวนี้มันสะท้อนความเป็นเผด็จการบ้านป่าด้อยพัฒ...
9,0,ตะกวดไทยมันยึดสูบเอง มันเห้สุดตีน


In [10]:
sample_df['text'][0]

'ไปเชียงใหม่ก็ระวังตัวกันเองนะครับ ต่อให้ขับมาแบบถูกกฏหมายแต่คนเชียงใหม่ขับเส้นทึบเยอะ เนี่ยสังคมเห็นแก่ตัว'

## Inference

In [11]:
PROMPT = f"""
Analyze the sentiment of the tweet determine if it is positive or negative, and return the answer as the corresponding sentiment label "positive" or  "negative" 

#response in only "positive" or "negative"
don't describe
"""

In [12]:
out = llm.create_chat_completion(
    messages = [
          {"role": "system", "content": PROMPT},
          {
              "role": "user",
              "content": sample_df['text'][0]
          }
      ],
    temperature=0.0
)

In [13]:
out

{'id': 'chatcmpl-7a96fc11-cb08-4961-8f87-ff4ee4dfb5a7',
 'object': 'chat.completion',
 'created': 1756963988,
 'model': '../quantize_model/llama3-8b-instruct-Q4K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'negative'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 116, 'completion_tokens': 1, 'total_tokens': 117}}

In [14]:
out['choices'][0]['message']['content']

'negative'

In [15]:
start_time = time.time()
output_list = []
for row in tqdm(sample_df['text'].tolist()):
    out = llm.create_chat_completion(
        messages = [
          {"role": "system", "content": PROMPT},
          {"role": "user", "content": row}
      ],
        temperature=0.0
    )
    output_list.append(out['choices'][0]['message']['content'])
end_time = time.time()
time_diff = end_time - start_time

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
output_list

['negative',
 'negative',
 'negative',
 'negative',
 '"negative"',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative']

### Change format

In [17]:
format_output = []
for row in output_list:
    if 'negative' in row:
        format_output.append(0)
    else:
        format_output.append(1)

In [18]:
format_output

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [19]:
sample_df['label'].tolist()

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]

## Evaluate

In [20]:
class_report = classification_report(y_true=sample_df['label'].tolist(), y_pred=format_output)

In [21]:
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      0.50      0.67         2

    accuracy                           0.90        10
   macro avg       0.94      0.75      0.80        10
weighted avg       0.91      0.90      0.89        10



In [22]:
print(f"Execution time: {time_diff:.6f} seconds")

Execution time: 0.381958 seconds
